In [49]:
#import modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import t
from sklearn.neighbors import KNeighborsRegressor

#import da
data = pd.read_csv('Trabalho.csv')
data.head(10)

,Unnamed: 0,Y,X1,X2,X3,X4
0,1,1.448136,0.291702,2.587011,2.564149,2.233988
1,2,3.001715,0.517189,3.462381,3.414562,-1.518295
2,3,3.168068,0.607613,4.052388,3.974191,1.875687
3,4,2.522207,1.615740,1.999990,1.880660,1.234311
4,5,0.085236,-0.992394,2.667726,2.611272,2.660299
5,6,3.293039,1.932170,3.196037,3.166793,0.232232
6,7,1.033810,0.044219,1.461748,1.343791,1.160812
7,8,2.695423,1.913302,2.205511,2.310261,-0.912108
8,9,3.626690,2.768036,2.038721,2.114006,-1.871051
9,10,2.611396,1.399541,1.239386,1.267726,0.136095


In [50]:
x =  data[['X1', 'X2', 'X3', 'X4']]
y = data['Y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)
mean_train = x_train.mean()
std_train = x_train.std()
print(mean_train)
print(std_train)
x_train_standardized = (x_train - mean_train) / std_train
print(x_train_standardized)

X1    1.065132
X2    2.135223
X3    2.140792
X4    1.004099
dtype: float64
X1    0.854632
X2    0.946178
X3    0.938934
X4    1.298703
dtype: float64
          X1        X2        X3        X4
11  0.414208 -0.129614 -0.107670  0.438472
47 -1.304123 -0.526428 -0.491542  2.070589
85 -1.283338  1.514198  1.425928 -0.524294
28  0.549892  1.663366  1.643894 -0.096866
93  0.484482 -1.517525 -1.603292  1.585748
..       ...       ...       ...       ...
60  0.177145  1.025887  0.881876 -1.203723
71  1.873699 -0.975415 -0.981258  0.441620
14  1.937073 -0.989189 -0.977106 -0.716054
92  0.921402  0.359304  0.578378  1.890191
51 -0.077770 -0.279738 -0.434346  0.100732

[70 rows x 4 columns]


In [51]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [52]:
y_pred = model.predict(x_test)
absolute_errors = np.abs(y_test - y_pred)
absolute_mean_error = np.mean(absolute_errors)
print(absolute_errors)
print(absolute_mean_error)

83    0.094474
53    0.806238
70    0.327541
45    0.562075
44    0.410855
39    0.270946
22    0.791341
80    0.509357
10    0.712092
0     0.628957
18    0.098713
30    0.356101
73    0.850269
33    0.407354
90    0.195628
4     0.887080
76    0.505140
77    0.411502
12    0.312403
31    0.309895
55    0.013837
88    0.379699
26    0.275539
42    0.311457
69    0.688510
15    0.674665
40    0.858351
96    0.178104
9     0.547023
72    0.156406
Name: Y, dtype: float64
0.4510517129704376


In [53]:
se_absolute_mean_error = np.std(absolute_errors) / np.sqrt(len(absolute_errors))
t_score = t.ppf(0.975, df = len(absolute_errors)-1)
print(t_score * se_absolute_mean_error)
lower_bound = absolute_mean_error - t_score * se_absolute_mean_error
upper_bound = absolute_mean_error + t_score * se_absolute_mean_error
print(f"Absolute Mean Error: {absolute_mean_error: .2f}")
print(f"95% Confidence Interval for Absolute Mean Error: [{lower_bound: .2f}, {upper_bound: .2f}]")

0.09188925706027758
Absolute Mean Error:  0.45
95% Confidence Interval for Absolute Mean Error: [ 0.36,  0.54]


In [54]:
k_values = [1, 3, 5]
knn_models = {}
knn_absolute_mean_errors = {}
knn_lower_bounds = {}
knn_upper_bounds = {}

for k in k_values:
    #create KNN regression
    model = KNeighborsRegressor(n_neighbors = k)
    model.fit(x_train, y_train)
    knn_models[k] = model
    
    y_pred = model.predict(x_test)
    absolute_errors = np.abs(y_test - y_pred)
    absolute_mean_error = np.mean(absolute_errors)
    knn_absolute_mean_errors[k] = absolute_mean_error
    
    se_absolute_mean_error = np.std(absolute_errors) / np.sqrt(len(absolute_errors))
    t_score = t.ppf(0.975, df = len(absolute_errors)-1)
    lower_bound = absolute_mean_error - t_score * se_absolute_mean_error
    upper_bound = absolute_mean_error + t_score * se_absolute_mean_error
    
    knn_lower_bounds[k] = lower_bound
    knn_upper_bounds[k] = upper_bound
    
for k in k_values:
    print(f"KNN (k={k}) Absolute Mean Error: {knn_absolute_mean_errors[k]: .2f}")
    print(f"KNN (k={k}) 95% Confidence Interval for Absolute Mean Error: [{knn_lower_bounds[k]:.2f}, {knn_upper_bounds[k]:.2f}]")

KNN (k=1) Absolute Mean Error:  0.66
KNN (k=1) 95% Confidence Interval for Absolute Mean Error: [0.42, 0.89]
KNN (k=3) Absolute Mean Error:  0.67
KNN (k=3) 95% Confidence Interval for Absolute Mean Error: [0.45, 0.89]
KNN (k=5) Absolute Mean Error:  0.70
KNN (k=5) 95% Confidence Interval for Absolute Mean Error: [0.45, 0.94]
